In [2]:
import json
import pickle as pkl
from pycparser import c_generator
import ForPragmaExtractor.visitors as visitor
from Model import tokenizer

VAR_PREFIX = "var"
ARR_PREFIX = "arr"
FUNC_PREFIX = "func"
STRUCT_PREFIX = "struct"
generator = c_generator.CGenerator()
id_v = visitor.CounterIdVisitor()
replacer = visitor.ReplaceIdsVisitor(VAR_PREFIX, ARR_PREFIX, STRUCT_PREFIX, FUNC_PREFIX)

In [3]:
def db_read_string_from_file(file_path):
    try:
        with open(file_path, "r") as f:
            return "".join(f.readlines())   # readlines()是读全部
    except:
        return ""


def normalize_code_as_ast(pickle_file):
    # print (pickle_file)
    with open(pickle_file, 'rb') as f:
        pragmafor_tuple = pkl.load(f)  #
        for_ast = pragmafor_tuple.for_node
        # for_ast.show()
        # print(normalize_code_as_string.generator.visit(for_ast))
        # for_ast.show()
        # counts in an array the name and identifiers of the code
        id_v.reset()
        id_v.visit(for_ast)
        # Replace the names..
        replacer.reset(id_v.ids, id_v.array,id_v.struct, id_v.func)
        replacer.visit(for_ast)
        with open('temp.txt', 'w') as f:
            for_ast.show(buf=f)
        with open('temp.txt', 'r') as f:
            ast = f.readlines()

        ast_no_whitespaces = [a.strip() for a in ast] # kill all whitespaces
        # print(ast_no_whitespaces)
        # print(normalize_code_as_string.generator.visit(for_ast))

        # print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
        ast_one_line = " " + " ".join(ast_no_whitespaces)
        return ast_one_line


# str1 = normalize_code_as_ast('DB_TEST/DB_TEST/database/PolyBench-ACC-master_.gitignore_2mm.c_5/code_pickle.pkl')
# print(str1)

def code_as_ast(pickle_file):
    # print (pickle_file)
    with open(pickle_file, 'rb') as f:
        pragmafor_tuple = pkl.load(f)  #
        for_ast = pragmafor_tuple.for_node
        with open('temp.txt', 'w') as f:
            for_ast.show(buf=f)
        with open('temp.txt', 'r') as f:
            ast = f.readlines()

        ast_no_whitespaces = [a.strip() for a in ast] # kill all whitespaces and \n
        ast_one_line = " " + " ".join(ast_no_whitespaces)
        return ast_one_line


str2 = code_as_ast('/Users/solochan/PycharmProjects/PragFormer/DB_TEST/database/PolyBench-ACC-master_.gitignore_2mm.c_5/code_pickle.pkl')
# print(str2)
# print('str2len: ', len(str2))

data1 = {'text': [str2]}
print("data1:\n", data1)

text, _ = tokenizer.deepscc_tokenizer(data1['text'])
print("after tokenized:\n", text)
print('token_len:', len(text.input_ids[0]), len(text.attention_mask[0]))

print("vocab_size: ", _)

data1:
 {'text': [" For: Assignment: = ID: i Constant: int, 0 BinaryOp: < ID: i ID: ni UnaryOp: p++ ID: i For: Assignment: = ID: j Constant: int, 0 BinaryOp: < ID: j ID: nl UnaryOp: p++ ID: j Assignment: = ArrayRef: ArrayRef: ID: D ID: i ID: j BinaryOp: / BinaryOp: * Cast: Typename: None, [] TypeDecl: None, [] IdentifierType: ['double'] ID: i BinaryOp: + ID: j Constant: int, 2 ID: nk"]}
after tokenized:
 {'input_ids': [[0, 286, 35, 46091, 35, 5457, 4576, 35, 939, 33685, 35, 6979, 6, 321, 47466, 13926, 35, 28696, 4576, 35, 939, 4576, 35, 10265, 1890, 1766, 13926, 35, 181, 42964, 4576, 35, 939, 286, 35, 46091, 35, 5457, 4576, 35, 1236, 33685, 35, 6979, 6, 321, 47466, 13926, 35, 28696, 4576, 35, 1236, 4576, 35, 295, 462, 1890, 1766, 13926, 35, 181, 42964, 4576, 35, 1236, 46091, 35, 5457, 42719, 31842, 35, 42719, 31842, 35, 4576, 35, 211, 4576, 35, 939, 4576, 35, 1236, 47466, 13926, 35, 1589, 47466, 13926, 35, 1009, 6719, 35, 5957, 9675, 4344, 35, 9291, 6, 48081, 7773, 45788, 35, 9291, 6, 

In [6]:
"""
    数据处理
"""
data_set = {'text':[], 'label':[], 'ast':[]}
jsonpath = '/Users/solochan/PycharmProjects/PragFormer/DB_TEST/database.json'
with open(jsonpath, 'r') as f:
    file_data = json.load(f)
    for i, key in enumerate(file_data):
        code = db_read_string_from_file(file_data[key]["code"])
        ast_str = code_as_ast(file_data[key]['code_pickle'])
        if file_data[key]['pragma']:
            pragma = db_read_string_from_file(file_data[key]['pragma'])
        else:
            pragma = '0'

        data_set['text'].append(code)
        data_set['ast'].append(ast_str)
        data_set['label'].append(pragma)

print("data_set['text'][0]\n", data_set['text'][0])
print("data_set['text'][10]\n", data_set['text'][10])
print("data_set['ast'][0]\n", data_set['ast'][0])
print("data_set['ast'][10]\n", data_set['ast'][10])
print("data_set['label'][0]\n", data_set['label'][0])
print("data_set['label'][10]\n", data_set['label'][10])

def label_encoder(labels:list):
    new_labels = []
    for label in labels:
        if 'omp parallel for' in label and 'private' not in label and 'reduction' not in label:
            new_labels.append('1')
        elif 'omp parallel for' in label and 'private' in label and 'reduction' not in label:
            new_labels.append('2')
        elif 'omp parallel for' in label and 'reduction' in label and 'private' not in label:
            new_labels.append('3')
        elif 'omp parallel for' in label and 'reduction' in label and 'private' in label:
            new_labels.append("4")
        else:
            new_labels.append("0")
    return new_labels


# def collate_fn(batch):
#     return {
#         'input_ids': pad_sequence([item['input_ids'] for item in batch], batch_first=True),
#         'attention_mask': pad_sequence([item['attention_mask'] for item in batch], batch_first=True),
#         'labels': torch.tensor([item['labels'] for item in batch])
#     }
#
print(data_set['label'])
data_set['label'] = label_encoder(data_set['label'])
print(data_set['label'])

data_set['text'][0]
 for (j = 0; j < prm->Natom; j++)
{
  for (i = 1; i < numcopies; i++)
  {
    sumdeijda[j] += sumdeijda[(prm->Natom * i) + j];
  }

}

data_set['text'][10]
 for (i = -1; i < prm->Natom; i++)
{
  iexw[eoff + i] = -1;
}

data_set['ast'][0]
  For: Assignment: = ID: j Constant: int, 0 BinaryOp: < ID: j StructRef: -> ID: prm ID: Natom UnaryOp: p++ ID: j Compound: For: Assignment: = ID: i Constant: int, 1 BinaryOp: < ID: i ID: numcopies UnaryOp: p++ ID: i Compound: Assignment: += ArrayRef: ID: sumdeijda ID: j ArrayRef: ID: sumdeijda BinaryOp: + BinaryOp: * StructRef: -> ID: prm ID: Natom ID: i ID: j
data_set['ast'][10]
  For: Assignment: = ID: i UnaryOp: - Constant: int, 1 BinaryOp: < ID: i StructRef: -> ID: prm ID: Natom UnaryOp: p++ ID: i Compound: Assignment: = ArrayRef: ID: iexw BinaryOp: + ID: eoff ID: i UnaryOp: - Constant: int, 1
data_set['label'][0]
 omp parallel for private(i, j) schedule(dynamic, blocksize)
data_set['label'][10]
 0
['omp parallel for private(i, 

In [8]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence



# 加载预训练的BERT模型和tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

# 数据集类
class CustomDataset(Dataset):
    def __init__(self, texts, asts, labels):
        self.texts = texts
        self.asts = asts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text_encoding = tokenizer(self.texts[idx], return_tensors='pt', padding=True, truncation=True, max_length=266)
        ast_encoding = tokenizer(self.asts[idx], return_tensors='pt', padding=True, truncation=True, max_length=266)
        label = torch.tensor(int(self.labels[idx]))

        return {
            'input_ids_text': text_encoding['input_ids'],
            'attention_mask_text': text_encoding['attention_mask'],
            'input_ids_ast': ast_encoding['input_ids'],
            'attention_mask_ast': ast_encoding['attention_mask'],
            'label': label
        }

# # 准备数据集
# dataset = CustomDataset(data_set['text'], data_set['ast'], data_set['label'])
# dataloader = DataLoader(dataset, batch_size=2, shuffle=True)
    
# 划分训练集和测试集
texts_train, texts_test, asts_train, asts_test, labels_train, labels_test = train_test_split(
    data_set['text'], data_set['ast'], data_set['label'], test_size=0.2, random_state=42
)

# 准备训练集和测试集的数据集
train_dataset = CustomDataset(texts_train, asts_train, labels_train)
test_dataset = CustomDataset(texts_test, asts_test, labels_test)


# 自定义 collate_fn 函数
def custom_collate_fn(batch):
    input_ids_text = [item['input_ids_text'] for item in batch]
    attention_mask_text = [item['attention_mask_text'] for item in batch]
    input_ids_ast = [item['input_ids_ast'] for item in batch]
    attention_mask_ast = [item['attention_mask_ast'] for item in batch]
    labels = torch.tensor([item['label'] for item in batch])

    # 使用 pad_sequence 进行填充
    input_ids_text = pad_sequence(input_ids_text, batch_first=True)
    attention_mask_text = pad_sequence(attention_mask_text, batch_first=True)
    input_ids_ast = pad_sequence(input_ids_ast, batch_first=True)
    attention_mask_ast = pad_sequence(attention_mask_ast, batch_first=True)

    # 将文本序列转换为可变长度序列
    lengths_text = torch.sum(attention_mask_text, dim=1)
    print('lengths_text shape: ', lengths_text.shape)
    packed_text = pack_padded_sequence(input_ids_text, lengths_text, batch_first=True, enforce_sorted=False)
    print('packed_text shape', packed_text.shape)

    # 将AST序列转换为可变长度序列
    lengths_ast = torch.sum(attention_mask_ast, dim=1)
    packed_ast = pack_padded_sequence(input_ids_ast, lengths_ast, batch_first=True, enforce_sorted=False)

    return {
        'packed_text': packed_text,
        'attention_mask_text': attention_mask_text,
        'packed_ast': packed_ast,
        'attention_mask_ast': attention_mask_ast,
        'label': labels
    }

# 创建 DataLoader 时使用 collate_fn 参数
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=custom_collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=False, collate_fn=custom_collate_fn)




# 定义优化器
optimizer = AdamW(model.parameters(), lr=5e-5)

# 训练模型
num_epochs = 3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        inputs_text = batch['input_ids_text'].to(device)
        attention_mask_text = batch['attention_mask_text'].to(device)
        inputs_ast = batch['input_ids_ast'].to(device)
        attention_mask_ast = batch['attention_mask_ast'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=inputs_text, attention_mask=attention_mask_text, inputs_ast=inputs_ast, attention_mask_ast=attention_mask_ast, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# 保存模型
# model.save_pretrained('your_model_directory')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/solochan/anaconda3/envs/Pragformer/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


RuntimeError: The size of tensor a (266) must match the size of tensor b (48) at non-singleton dimension 1

In [ ]:
# 测试模型
model.eval()
all_predictions = []
all_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        inputs_text = batch['input_ids_text'].to(device)
        attention_mask_text = batch['attention_mask_text'].to(device)
        inputs_ast = batch['input_ids_ast'].to(device)
        attention_mask_ast = batch['attention_mask_ast'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids=inputs_text, attention_mask=attention_mask_text,
                        inputs_ast=inputs_ast, attention_mask_ast=attention_mask_ast)

        predictions = torch.argmax(outputs.logits, dim=1).cpu().numpy()
        all_predictions.extend(predictions)
        all_labels.extend(labels.cpu().numpy())

# 计算测试集准确率
accuracy = accuracy_score(all_labels, all_predictions)
print(f'Test Accuracy: {accuracy}')

分割线
--------------------------------------------


In [25]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
from torch.nn.utils.rnn import pad_sequence



In [26]:
# Define your dataset class
class MyDataset(Dataset):
    def __init__(self, texts, asts, labels, tokenizer, max_len):
        self.texts = texts
        self.asts = asts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        ast = str(self.asts[idx])
        label = int(self.labels[idx])

        inputs = self.tokenizer(text, ast, return_tensors='pt', max_length=self.max_len, truncation=True, padding='max_length')
        inputs['labels'] = torch.tensor(label)
        # print('mydata: ', inputs)
        return inputs

# Encode labels 这种编码不适合当前的任务
# label_encoder = LabelEncoder()
# data_set['label'] = label_encoder.fit_transform(data_set['label'])
# print('\n', data_set['label'])

def label_encoder(labels:list):
    new_labels = []
    for label in labels:
        if 'omp parallel for' in label and 'private' not in label and 'reduction' not in label:
            new_labels.append('1')
        elif 'omp parallel for' in label and 'private' in label and 'reduction' not in label:
            new_labels.append('2')
        elif 'omp parallel for' in label and 'reduction' in label and 'private' not in label:
            new_labels.append('3')
        elif 'omp parallel for' in label and 'reduction' in label and 'private' in label:
            new_labels.append("4")
        else:
            new_labels.append("0")
    return new_labels


# def collate_fn(batch):
#     return {
#         'input_ids': pad_sequence([item['input_ids'] for item in batch], batch_first=True),
#         'attention_mask': pad_sequence([item['attention_mask'] for item in batch], batch_first=True),
#         'labels': torch.tensor([item['labels'] for item in batch])
#     }
#
print(data_set['label'])
data_set['label'] = label_encoder(data_set['label'])
print(data_set['label'])

# Split the dataset
train_texts, val_texts, train_asts, val_asts, train_labels, val_labels = train_test_split(
    data_set['text'], data_set['ast'], data_set['label'], test_size=0.2, random_state=42
)

# Define tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

# Define datasets and dataloaders
train_dataset = MyDataset(train_texts, train_asts, train_labels, tokenizer, max_len=512)
val_dataset = MyDataset(val_texts, val_asts, val_labels, tokenizer, max_len=512)

print('train_dataset: ', train_dataset)
print('val_dataset: ', val_dataset)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# Define training parameters
optimizer = AdamW(model.parameters(), lr=1e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)



['omp parallel for private(i, j) schedule(dynamic, blocksize)', 'omp parallel for private(i, j) schedule(dynamic, blocksize)', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'omp parallel for schedule(dynamic,4) shared(status)', 'omp parallel for schedule(dynamic,4) shared(status)', 'omp parallel for schedule(dynamic,4) shared(status)', 'omp parallel for schedule(dynamic,4) shared(status)', 'omp parallel for schedule(dynamic,4) shared(status)', '0', '0', '0', '0', '0', '0', '0', 'omp parallel for schedule(dynamic,4) shared(status)', 'omp parallel for schedule(dynamic,4) shared(status)', 'omp parallel for schedule(dynamic,4) shared(status)', '0', '0', 'omp parallel for schedule(dynamic,4) shared(status)', 'omp parallel for schedule(dynamic,4) shared(status)', 'omp parallel for schedule(dynamic,4) shared(status)', 'omp parallel for schedule(dynamic,4) shared(status)', 'omp parallel for schedule(dynamic,4) shared(status)', 'omp parallel for sched

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


train_dataset:  <__main__.MyDataset object at 0x17f879f30>
val_dataset:  <__main__.MyDataset object at 0x1691b3d90>


/Users/solochan/anaconda3/envs/Pragformer/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
print(train_dataloader)
for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(train_dataloader):
    print("input_ids: ", input_ids)
    print("attention_mask: ", attention_mask)
    print("token_type_ids: ", token_type_ids)
    

In [ ]:
# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}"):
        inputs = {key: value.to(device) for key, value in batch.items()}
        print("inputs: \n", inputs)
        # print('training inputs: ', inputs)
    #     optimizer.zero_grad()
    #     outputs = model(**inputs)
    #     loss = outputs.loss
    #     loss.backward()
    #     optimizer.step()

    # # Validation
    # model.eval()
    # val_preds = []
    # val_true = []
    # with torch.no_grad():
    #     for batch in tqdm(val_dataloader, desc=f"Epoch {epoch + 1} - Validation"):
    #         inputs = {key: value.to(device) for key, value in batch.items()}
            
    #         outputs= model(**inputs)
    #         logits = outputs.logits
    #         preds = torch.argmax(logits, dim=1)
    #         val_preds.extend(preds.cpu().numpy())
    #         val_true.extend(inputs['label'].cpu().numpy())

    # # Calculate validation accuracy and print metrics
    # accuracy = accuracy_score(val_true, val_preds)
    # report = classification_report(val_true, val_preds, target_names=label_encoder.classes_)
    # print(f"Epoch {epoch + 1} - Validation Accuracy: {accuracy:.4f}")
    # print(report)